* Cycle Detection : check the figure
* Topological Sort: keep the left figure
* Eulerian Circuit (optional)

To do list
* Each section should have a fun application with better figure!!!
* the disjoint set can be improved using tree: https://www.hackerearth.com/practice/algorithms/graphs/minimum-spanning-tree/tutorial/
* Proof to dijkstra should be a practice of the proof of greedy algorithms
* Add more leetcode examples.

Resources
* https://web.stanford.edu/class/archive/cs/cs161/cs161.1138/lectures/14/Small14.pdf
https://homes.luddy.indiana.edu/achauhan/Teaching/B403/LectureNotes/09-mst.html

### Cycle Detection

In [1]:
# initialization
class STATE:
  white = 0
  gray = 1
  black = 2

#### Directed graph

In [61]:
dcg = [[1], [2], [0, 4], [], [3], [6], []]

In [75]:
def hasCycleDirected(g, s, state):
  state[s] = STATE.gray # first be visited
  for v in g[s]:
    if state[v] == STATE.white:
      if hasCycleDirected(g, v, state):
        print(f'Cycle found at node {v}.')
        return True
    elif state[v] == STATE.gray: # aback edge
      print(f'Cycle starts at node {v}.')
      return True
    else:
        pass
    state[s] = STATE.black # mark it as complete
    return False

In [63]:
def cycleDetectDirected(g):
  n = len(g)
  state = [STATE.white] * n
  for i in range(n):
    if state[i] == STATE.white:
      if hasCycleDirected(g, i, state):
        return True
  return False

In [64]:
cycleDetectDirected(dcg)

Cycle starts at node 0.
Cycle found at node 2.
Cycle found at node 1.


True

In [77]:
# run one with a forward edge, change (2, 0) to (0, 2)
dag = [[1, 2], [2], [4], [], [3], [6], []]


In [78]:
cycleDetectDirected(dag)

False

#### Undirected Graph

In [98]:
def hasCycleUndirected(g, s, p, visited):
  visited[s] = True
  for v in g[s]:
      if not visited[v]:
        if hasCycleUndirected(g, v, s, visited):
          print(f'Cycle found at node {v}.')
          return True
      else:
        if v != p: # both black and gray
          print(f'Cycle starts at node {v}.')
          print(visited[v])
          return True

  return False

In [99]:
def cycleDetectUndirected(g):
  n = len(g)
  visited = [False] * n
  for i in range(n):
    if not visited[i]:
      if hasCycleUndirected(g, i, -1, visited):
        print(f'Cycle found at start node {i}.')
        return True

  return False

In [100]:
ucg=[[1, 2], [0, 2], [0, 4], [], [2, 3], [6], [5]]

In [101]:
cycleDetectUndirected(ucg)

Cycle starts at node 0.
True
Cycle found at node 2.
Cycle found at node 1.
Cycle found at start node 0.


True

In [27]:
# delete edge (0, 2)
uag = [[1], [0, 2], [4], [], [2, 3], [6], [5]]
cycleDetectUndirected(uag)

False

# Topolgical Sort

In [28]:
# Directed Acyclic Graph 
dag = [[1], [2], [], [2, 4, 5], [], [6], []]
dag

[[1], [2], [], [2, 4, 5], [], [6], []]

In [29]:
# Directed Cyclic Graph
dcg = [[1], [2], [3], [2, 4, 5], [], [6], []]
dcg

[[1], [2], [3], [2, 4, 5], [], [6], []]

## Kahn's algorithm

In [30]:
from collections import defaultdict
import heapq 
def kahns_topo_sort(g):
  S = []
  V_S =[(0, node) for node in range(len(g))] # initialize node with 0 as in-degree
  indegrees = defaultdict(int)
  # Step 1: count the in-degree
  for u in range(len(g)):
    indegrees[u] = 0
  for u in range(len(g)):
    for v in g[u]:
      indegrees[v]+= 1
  print(f'initial indegree : {indegrees}')
  V_S = [(indegree, node) for node, indegree in indegrees.items()]
  heapq.heapify(V_S)

  # Step 2: Kan's algorithm
  while len(V_S) > 0:
    indegree, first_node = V_S.pop(0)
    if indegree != 0: # cycle found, no topological ordering
      print(f'Cycle starts at {first_node}')
      return None
    S.append(first_node)
    # Remove edges
    for v in g[first_node]:
      indegrees[v] -= 1
    # update V_S
    for idx, (indegree, node) in enumerate(V_S):
      if indegree != indegrees[node]:
        V_S[idx] = (indegrees[node], node)
    heapq.heapify(V_S)
  return S

In [31]:
kahns_topo_sort(dag)

initial indegree : defaultdict(<class 'int'>, {0: 0, 1: 1, 2: 2, 3: 0, 4: 1, 5: 1, 6: 1})


[0, 1, 3, 2, 4, 5, 6]

In [32]:
kahns_topo_sort(dcg)

initial indegree : defaultdict(<class 'int'>, {0: 0, 1: 1, 2: 2, 3: 1, 4: 1, 5: 1, 6: 1})
Cycle starts at 2


## DFS

In [43]:
# def dfs(g, s, colors, complete_orders):
#   colors[s] = STATE.gray
#   no_cycle = True
#   for v in g[s]:
#     if colors[v] == STATE.white:
#       no_cycle = no_cycle and dfs(g, v, colors, complete_orders)
#     elif colors[v] == STATE.gray: # a cycle appears
#       print(f'Cycle found at node {v}.')
#       return False
#   colors[s] = STATE.black
#   complete_orders.append(s)
#   return no_cycle

def dfs(g, s, colors, complete_orders):
  colors[s] = STATE.gray
  for v in g[s]:
    if colors[v] == STATE.white:
      if dfs(g, v, colors, complete_orders):
        return True
    elif colors[v] == STATE.gray: # a cycle appears
      print(f'Cycle found at node {v}.')
      return True
  colors[s] = STATE.black
  complete_orders.append(s)
  return False

In [44]:
def topo_sort(g):
  n = len(g)
  complete_orders = []
  colors = [STATE.white] * n
  for i in range(n): # run dfs on all the node
    if colors[i] == STATE.white:
      if dfs(g, i, colors, complete_orders):
        print('Cycle found, no topological ordering')
        return None 
  return complete_orders[::-1]

In [45]:
topo_sort(dag) # [3, 5, 6, 4, 0, 1, 2]


[3, 5, 6, 4, 0, 1, 2]

In [46]:
topo_sort(dcg)

Cycle found at node 2.
Cycle found, no topological ordering


# Connected Components

In the example, we only experiment with BFS. You can use DFS too.

## Undirected Graph

### Graph search approach

In [115]:
ug = [[1, 2], [0, 2], [0, 4], [], [2, 3], [6], [5]]

In [116]:
def bfs(g, s, state):
  state[s] = True
  
  q, orders = [s], [s]
  while q:
    u = q.pop(0)
    for v in g[u]:
      if not state[v]:
        state[v] = True
        q.append(v)
        orders.append(v)
  return orders

In [117]:
def connectedComponent(g):
  n = len(g)
  ccs = []
  state = [False] * n
  for i in range(n):
    if not state[i]:
      ccs.append(bfs(g, i, state))
  return ccs    
  

In [118]:
connectedComponent(ug)

[[0, 1, 2, 4, 3], [5, 6]]

### Union-find approach



Use disjoint set union: https://cp-algorithms.com/data_structures/disjoint_set_union.html

http://www.cs.utexas.edu/users/djimenez/utsa/cs3343/lecture18.html

In [119]:
# implement disjoint set with tree based set representative and path compression
class DisjointSet:
    def __init__(self, n):
        self.n = n
        self.p = [i for i in range(n)]

    def find(self, x):
        if self.p[x] != x:
            self.p[x] = self.find(self.p[x])
        return self.p[x]

    def union(self, x, y):
        xr = self.find(x)
        yr = self.find(y)
        self.p[xr] = yr
    
    # Add two more functions to visualize the sets
    def get_num_sets(self):
       return sum(self.find(i) == i for i in range(self.n))

    def get_all_sets(self):
      sets = defaultdict(set)
      for i in range(self.n):
        p = self.find(i)
        sets[p].add(i)
      return sets

In [120]:
# implement union-find based connect component detection
from collections import defaultdict
def connectedComponent(g):
  n = len(g)
  # initialize disjoint set
  ds = DisjointSet(n)

  for i in range(n):
    for j in g[i]: # for edge i<->j
      ds.union(i, j)
  return ds.get_num_sets(), ds.get_all_sets() 

In [121]:
connectedComponent(ug)

(2, defaultdict(set, {3: {0, 1, 2, 3, 4}, 6: {5, 6}}))

Dynamic graph

In [122]:
# represent the graph with a list of edges
ug_edges = [(0, 1), (0, 2), (1, 2), (2, 4), (4, 3), (4, 3), (5, 6)]

In [123]:
class DynamicConnectedComponent():
  def __init__(self):
    self.ds = DisjointSet(0)
    self.node_index= defaultdict(int)
    self.index_node = defaultdict(int)
    self.index = 0

  def add_edge(self, u, v):
    if u not in self.node_index:
      self.node_index[u], self.index_node[self.index] = self.index, u
      self.ds.p.append(self.index)
      self.ds.n += 1
      self.index += 1
      
    if v not in self.node_index:
      self.node_index[v], self.index_node[self.index] = self.index, v
      self.ds.p.append(self.index)
      self.ds.n += 1
      self.index += 1
    u, v = self.node_index[u], self.node_index[v]
    self.ds.union(u, v)
    return

  def get_num_sets(self):
    return self.ds.get_num_sets()

  def get_all_sets(self):
    sets = self.ds.get_all_sets()
    return {self.index_node[key] : set([self.index_node[i] for i in list(value)]) for key, value in sets.items()} 

In [124]:
dcc = DynamicConnectedComponent()
for u, v in ug_edges: 
  dcc.add_edge(u, v)

dcc.get_num_sets(), dcc.get_all_sets()

(2, {3: {0, 1, 2, 3, 4}, 6: {5, 6}})

## Directed Graph and SCCs
https://web.stanford.edu/class/archive/cs/cs161/cs161.1138/lectures/04/Small04.pdf

In [36]:
dg = [[2], [0], [1, 3], [2, 4], [5], [6], []]

In [37]:
# DFS traversal with reversed complete orders
def dfs(g, s, colors, complete_orders):
  colors[s] = STATE.gray
  for v in g[s]:
    if colors[v] == STATE.white:
      dfs(g, v, colors, complete_orders)
  colors[s] = STATE.black
  complete_orders.append(s)
  return

In [38]:
# topologically sort in terms of the last node of each scc
def topo_sort_scc(g):
  v = len(g)
  complete_orders = []
  colors = [STATE.white] * v
  for i in range(v): # run dfs on all the node
    if colors[i] == STATE.white:
      dfs(g,i, colors, complete_orders)
  return complete_orders[::-1]

In [39]:
topo_sort_scc(dg)

[0, 2, 3, 4, 5, 6, 1]

In [40]:
# get conversed graph
def reverse_graph(g):
  rg = [[] for i in range(len(g))]
  for u in range(len(g)):
    for v in g[u]:
      rg[v].append(u)
  return rg

In [41]:
def scc(g):
  rg = reverse_graph(g)
  orders = topo_sort_scc(g)

  # track states
  colors = [STATE.white] * len(g)
  sccs = []

  # traverse the reversed graph
  for u in orders:
    if colors[u] != STATE.white:
      continue
    scc = []
    dfs(rg, u, colors, scc)
    sccs.append(scc)
  return sccs

In [42]:
scc(dg)

[[3, 2, 1, 0], [4], [5], [6]]

# Minimum Spanning Tree

https://web.stanford.edu/class/archive/cs/cs106b/cs106b.1138/lectures/24/Slides24.pdf

## Kruskal's Algorithm

In [132]:
from typing import Dict
def kruskal(g: Dict):
  # g is a dict with node: adjacent nodes
  vertices = [i for i in range(1, 1 + len(g))]
  vertices = g.keys()
  n  = len(vertices)
  ver_idx = {v: i for i, v in enumerate(vertices)}

  # initialize a disjoint set
  ds = DisjointSet(n)

  # sort all edges
  edges = []
  for u in vertices:
    for v, w in g[u]:
      if (v, u, w) not in edges:
        edges.append((u, v, w))
  edges.sort(key=lambda x: x[2])
  
  # main section
  A = []
  for u, v, w in edges:
    if ds.find(ver_idx[u]) != ds.find(ver_idx[v]):
      ds.union(ver_idx[u], ver_idx[v])
      print(f'{u} -> {v}: {w}')
      A.append((u, v, w))
  return A

In [133]:
a= {1:[(2, 2), (3, 12)], 2:[(1, 2), (3, 4), (5, 5)], 3:[(1, 12), (2, 4), (4, 6), (5, 3)], 4:[(3, 6),  (5, 7)], 5:[(2, 5), (3, 3), (4, 7)]}
kruskal(a)

1 -> 2: 2
3 -> 5: 3
2 -> 3: 4
3 -> 4: 6


[(1, 2, 2), (3, 5, 3), (2, 3, 4), (3, 4, 6)]

## Prim's Algorithm

Priority queue by edges

In [134]:
import queue

def _get_light_edge(pq, S):
  while pq:
    # Pick the light edge
    w, u, v = pq.get()
    # Filter out non-cross edge
    if v not in S:
      S.add(v)
      return (u, v, w)
  return None
  
def prim(g):
  cur = 1
  n = len(g.items())
  S = {cur} #spanning tree set
  pq = queue.PriorityQueue()
  A = []
  
  while len(S) < n:
    # Expand edges for the exploring vertex
    for v, w in g[cur]:
      if v not in S:
        pq.put((w, cur, v))

    le = _get_light_edge(pq, S)
    if le:
      A.append(le)
      cur = le[1] #set the exploring vertex
    else:
      print(f'Graph {g} is not connected.')
      break
  return A 

In [135]:
prim(a)

[(1, 2, 2), (2, 3, 4), (3, 5, 3), (3, 4, 6)]

priority queue by vertices

In [136]:
from heapq import heappush, heappop, heapify
from typing import List
import itertools
class PriorityQueue:
  def __init__(self, items:List[List]=[]):
      self.pq = []                         # list of entries arranged in a heap
      self.entry_finder = {}               # mapping of tasks to entries
      self.REMOVED = '<removed-task>'      # placeholder for a removed task
      self._counter = itertools.count()     # unique sequence count, this is hidden from user
      # add count to items
      for p, t, info in items:
        item = [p, next(self._counter), t, info]
        self.entry_finder[t] = item
        self.pq.append(item)
      heapify(self.pq)
        
  def add_task(self, task, priority=0, info=None): # O(logE)
      'Add a new task or update the priority of an existing task'
      'the old task is removed from entry_finder but still remained in the heapq'
      if task in self.entry_finder:
          self._remove_task(task)
      count = next(self._counter)
      entry = [priority, count, task, info]
      self.entry_finder[task] = entry
      heappush(self.pq, entry)
      
  def _remove_task(self, task, info=None):# O(1)
      'Mark an existing task as REMOVED.  Raise KeyError if not found.'
      entry = self.entry_finder.pop(task)
      entry[-2] = self.REMOVED

  def pop_task(self): #O(logE)
      'Remove and return the lowest priority task. Raise KeyError if empty.'
      while self.pq:
          priority, count, task, info = heappop(self.pq)
          if task is not self.REMOVED:
              del self.entry_finder[task]
              return task, info, priority
      raise KeyError('pop from an empty priority queue')
      
  def get_task(self, taskid):
    '''return task information given task id'''
    if taskid in self.entry_finder:
      p, _, t, info = self.entry_finder[taskid]
      return p, info
    else:
      return None, None
  
  def empty(self):
    return not self.entry_finder

In [137]:
def prim2(g):  
  n = len(g.items())
  pq = PriorityQueue()
  S = {}
  A = []
  # Initialization
  for i in range(n):
    pq.add_task(task=i+1, priority=float('inf'), info=None) # task: vertex, priority: edge cost, info: predecessor vertex
  
  S = {1}
  pq.add_task(1, 0, info=1)

  while len(S) < n:
    u, p, w = pq.pop_task()
    if w == float('inf'):
      print(f'Graph {g} is not connected.')
      break
    A.append((p, u, w))
    S.add(u)
    for v, w in g[u]:
      if v not in S and  w < pq.entry_finder[v][0]:
        pq.add_task(v, w, u)
  
  return A


In [138]:
print(prim2(a))

[(1, 1, 0), (1, 2, 2), (2, 3, 4), (3, 5, 3), (3, 4, 6)]


# Shortest Path


1.   brute force solution
2.   illustrating the overleapping subproblem
3.   illustrating the optimal subproblems
4.   Introduce bellman-ford algorithm
5.   introduce dijkstra algorithm

Questions:
1. detect the negative weight cycle




#### Single Source Shortest Path

In [139]:
import sys
from collections import defaultdict
g = {
    's':[('t', 6), ('y', 7)],
    't':[('x', 5), ('y', 8), ('z', -4)],
    'x':[('t',-2)],
    'y':[('x',-3), ('z',9)],
    'z':[('x',7)]    
}

g1 = {
    's':[('t', 6), ('y', 7)],
    't':[('x', -5), ('y', 8), ('z', -4)],
    'x':[('t',2)],
    'y':[('x',-3), ('z',9)],
    'z':[('x',7)]    
}

Backtracking to enumerate all paths from start vertex and its cost. The following code works for both undirected and directed graph.



In [140]:
def all_paths(g, s, path, cost, ans):
  ans.append({'path': path[::], 'cost': cost})
  for v, w in g[s]:
    if v in path:
      continue
    path.append(v)
    cost += w
    all_paths(g, v, path, cost, ans)
    cost -= w
    path.pop()

In [141]:
def get_shortest_path(ans, g):
  delta = {v: float('inf') for v in g.keys()} # shortest path value from s to all vertices
  delta_path = {v: [] for v in g.keys()}
  for item in ans:
    path = item['path']
    cost = item['cost']
    target = path[-1]
    if cost < delta[target]:
      delta[target] = cost
      delta_path[target] = path
  return delta, delta_path

In [142]:
ans, path, cost = [], ['s'], 0
all_paths(g, 's', path, cost, ans  )
get_shortest_path(ans, g)

({'s': 0, 't': 2, 'x': 4, 'y': 7, 'z': -2},
 {'s': ['s'],
  't': ['s', 'y', 'x', 't'],
  'x': ['s', 'y', 'x'],
  'y': ['s', 'y'],
  'z': ['s', 'y', 'x', 't', 'z']})

In [143]:
# on graph 2
ans1, path, cost = [], ['s'], 0
all_paths(g1, 's', path, cost, ans1  )
get_shortest_path(ans1, g1)

({'s': 0, 't': 6, 'x': 1, 'y': 7, 'z': 2},
 {'s': ['s'],
  't': ['s', 't'],
  'x': ['s', 't', 'x'],
  'y': ['s', 'y'],
  'z': ['s', 't', 'z']})

## The Bellman-Ford Algorithm
Use the dp vector and W

In [144]:
g = {
    's':[('t', 6), ('y', 7)],  
    't':[('x', 5), ('y', 8), ('z', -4)],
    'x':[('t',-2)],
    'y':[('x',-3), ('z',9)],
    'z':[('x',7)],
}

In [145]:
# convert g to W
n = len(g)
W = [[float('inf') for _ in range(n)] for _ in range(n)]
# Assign an enumerial index for each key
V = g.keys()
# Key : index
ver2idx = dict(zip(V, [i for i in range(n)]))
# Index : key
idx2ver = dict(zip([i for i in range(n)], V))
print(f'ver2idx : {ver2idx}')
print(f'idx2ver :{idx2ver}')
for u in V:
  ui = ver2idx[u]
  W[ui][ui] = 0
  for v, w in g[u]:
    vi = ver2idx[v]
    W[ui][vi] = w
W

ver2idx : {'s': 0, 't': 1, 'x': 2, 'y': 3, 'z': 4}
idx2ver :{0: 's', 1: 't', 2: 'x', 3: 'y', 4: 'z'}


[[0, 6, inf, 7, inf],
 [inf, 0, 5, 8, -4],
 [inf, -2, 0, inf, inf],
 [inf, inf, -3, 0, 9],
 [inf, inf, 7, inf, 0]]

In [146]:
## using W
def bellman_ford_dp(s, W):
  n = len(W)
  # D, pi
  D = [float('inf') if i!=s else 0 for i in range(n)] # * n
  P = [None] * n
  for m in range(n-1): 
    newD = D[:]
    for i in range(n): # endpoint
      for k in range(n): # intermediate node
        if D[k] + W[k][i] < newD[i]:
          P[i] = k
          newD[i] = D[k] + W[k][i]

    D = newD
    print(f'D{m+1}: {D}')
  return D, P

In [147]:
# print the path from s-> u through backtracking starting from u
def get_path(P, s, u, path):
  path.append(u)
  if u == s:
    print('Reached to the source vertex, stop!')
    return path[::-1]
  elif u is None:
    print(f"No path found between {s} and {u}.")
    return [] 
  else:  
    return get_path(P, s, P[u], path)

In [148]:
D, P = bellman_ford_dp(0, W)
D, P

D1: [0, 6, inf, 7, inf]
D2: [0, 6, 4, 7, 2]
D3: [0, 2, 4, 7, 2]
D4: [0, 2, 4, 7, -2]


([0, 2, 4, 7, -2], [None, 2, 3, 0, 1])

In [149]:
get_path(P, 0, 4, [])

Reached to the source vertex, stop!


[0, 3, 2, 1, 4]

Formal bellman-ford algorithm

In [150]:
g = {
    's':[('t', 6), ('y', 7)],  
    't':[('x', 5), ('y', 8), ('z', -4)],
    'x':[('t',-2)],
    'y':[('x',-3), ('z',9)],
    'z':[('x',7)],
}

In [151]:
def bellman_ford(g: dict, s: str):
  n = len(g)
  # Assign an enumerial index for each key
  V = g.keys()
  # Key to index
  ver2idx = dict(zip(V, [i for i in range(n)]))
  # Index to key
  idx2ver = dict(zip([i for i in range(n)], V))
  # Initialization the dp matrix with d estimate and predecessor
  si = ver2idx[s]
  D = [float('inf') if i!=si else 0 for i in range(n)] # * n
  P = [None] * n
  
  # n-1 passes
  for i in range(n-1): 
    # relax all edges
    for u in V:
      ui = ver2idx[u]
      for v, w in g[u]:
        vi = ver2idx[v]
        # Update dp's minimum path value and predecessor
        if D[vi] > D[ui] + w:
          D[vi] = D[ui] + w
          P[vi] = ui
    print(f'D{i+1}: {D}')  
  return D, P, ver2idx, idx2ver

In [152]:
bellman_ford(g, 's')

D1: [0, 6, 4, 7, 2]
D2: [0, 2, 4, 7, 2]
D3: [0, 2, 4, 7, -2]
D4: [0, 2, 4, 7, -2]


([0, 2, 4, 7, -2],
 [None, 2, 3, 0, 1],
 {'s': 0, 't': 1, 'x': 2, 'y': 3, 'z': 4},
 {0: 's', 1: 't', 2: 'x', 3: 'y', 4: 'z'})

Implement a version that plots the process, and use HTML-like to draw the graph with math symbols: https://stackoverflow.com/questions/9684807/how-to-insert-mathematical-symbols-like-greek-characters-in-a-graphviz-dot-file/41346289#41346289

Optimization with topologial sort

In [48]:
# dag and the ordering is topological sorted order already
dag = {
    's':[('t', 6), ('y', 7)], 
    't':[('x', 5), ('y', 8), ('z', -4)],
    'y':[('x',-3), ('z',9)],
    'z':[('x',7)],
    'x':[],  
}

Linear Bellman-ford algorithm for DAG.

In [47]:
def bellman_ford_dag(g, s):
  s = s
  n = len(g)
  # Key to index
  ver2idx = dict(zip(g.keys(), [i for i in range(n)]))
  # Index to key
  idx2ver = dict(zip([i for i in range(n)], g.keys()))
  print(ver2idx)
  # Convert g to index
  ng = [[] for _ in range(n)]
  for u in g.keys():
    for v, _ in g[u]:
      ui = ver2idx[u]
      vi = ver2idx[v]
      ng[ui].append(vi)
  print(ng)
  V = topo_sort(ng)
  print(V)
  # Initialization the dp matrix with d estimate and predecessor
  si = ver2idx[s]
  dp = [(float('inf'), None) for i in range(n)]
  dp[si] = (0, None)

  # relax all edges
  for ui in V:
    u = idx2ver[ui]
    for v, w in g[u]:
      vi = ver2idx[v]
      # Update dp's minimum path value and predecessor
      if dp[vi][0] > dp[ui][0] + w:
        dp[vi] = (dp[ui][0] + w, ui)
  return dp

In [50]:
bellman_ford_dag(dag, 's')

{'s': 0, 't': 1, 'y': 2, 'z': 3, 'x': 4}
[[1, 2], [4, 2, 3], [4, 3], [4], []]
[0, 1, 2, 3, 4]


[(0, None), (6, 0), (7, 0), (2, 1), (4, 2)]

## Dijkstra

In [164]:
g = {
    's':[('t', 6), ('y', 7)], 
    't':[('x', 5), ('y', 8), ('z', 4)],
    'y':[('x', 3), ('z', 9)],
    'z':[('x', 7)],
    'x':[('t', 2)],  
}

g = {
    '0':[('1', 50), ('2', 30), ('3', 100), ('4', 10)], 
    '1':[('0', 50), ('2', 5), ('3', 20)],
    '2':[('1', 5), ('3', 6), ('0', 30)],
    '3':[('0', 100), ('1', 20), ('2', 6), ('4', 20)],
    '4':[('0', 10),('3', 20)],  
}



In [ ]:
g

In [ ]:
plot_graph(g, name="directed_nonnegative_graph")

In [ ]:

from typing import List, Tuple
import sys

def initialize(Q, g, s):
  # put all edges into q
  for k in g.keys():
    Q.add_task(task=k, priority=sys.maxsize, info=None ) #task id is the id, info is the predecessor
  # set weight for vertices one edge away from source
  Q.add_task(task=s, priority=0, info=None)
  for id, w in g[s]:
    Q.add_task(task=id, priority=w, info=s) # weight, id, predecessor

      
def extract_min(Q:List[Tuple]):
  '''extra minimum vertex with the weight using priority queue'''
  task, info, pri = Q.pop_task()
  return task, pri, info

def update(Q, g, cid, cw):
  '''current id, w, p'''
  for id, w in g[cid]: #target
    pw, pp = Q.get_task(id)
    if not pw and not pp: # already found the shortest path for this id
      continue
    new_w, new_p = w, pp
    #print(cw, w, pw)
    if cw + w < pw:
      new_w = cw + w
      new_p = cid
      Q.add_task(task=id, priority=new_w, info=new_p)
  
def dijkstra(g, s):
  ''' '''
  Q = PriorityQueue() # the set of all edges, 
  S = []
  # initialize
  initialize(Q, g, s)
  while not Q.empty():
    min_id, min_w, p = extract_min(Q)
    print(min_id, min_w, p)
    S.append((min_id, min_w, p))
    # need to update weight in the queue 
    update(Q, g, min_id, min_w)
  return S

In [ ]:
S=dijkstra(g, '0')
print(S)

In [ ]:
def dijkstra(g, s):
  Q = PriorityQueue()
  S = []
  # task: vertex id, priority: shortest-path estimate, info: predecessor
  Q.add_task(task=s, priority=0, info=None)
  visited = set()
  while not Q.empty():
    # Use the light vertex
    u, up, ud = Q.pop_task()
    visited.add(u)
    S.append((u, ud, up))

    # Relax adjacent vertice
    for v, w in g[u]: 
      # Already found the shortest path for this id
      if v in visited: 
        continue
      
      vd, vp = Q.get_task(v)
      # First time to add the task or already in the queue, but need update 
      if not vd  or ud + w < vd:
        Q.add_task(task=v, priority=ud + w, info=u)
  return S

In [ ]:
dijkstra(g, 's')

Get visualization

In [ ]:
def plot_subgraph(dot, v_pos, g, Q, index):
  #dot = Digraph(comment='directed_graph', engine="neato", format='png', strict=True, )
  dot.attr(_attributes={'fontsize': '15'})
  name = ''
  if index == 0:
    for u, p in v_pos.items(): #label shows in the node, it is also what represents each node
      x, y = p.split(',')
      #p = str(int(x)+5*(index+1)) + ',' + y
      xlabel = '<&infin;>'
      # label is the one showing within the node, and name act as the name used in the edge, and xlabel is the tag above each node
      dot.node(name=u+name, label=xlabel, xlabel = u, _attributes={'pos': str(p), 'penwidth': "1.5"})
      for v, w in g[u]:
        dot.edge(tail_name=u+name, head_name=v+name, _attributes={'xlabel': str(w), 'penwidth': "1.5"})

  for vd, _, v, u in Q.pq:
    if v == Q.REMOVED:
      continue
    dot.node(name=v+name, label=str(vd), xlabel=v, _attributes={'color': 'red', 'penwidth': "1.5"})
    if u is not None:
      dot.edge(u+name, v+name, _attributes={'color': 'red', 'penwidth': "1.5" })
    #init.view()
  dot.render(f'test-output/dijkstra_{index}', view=True, format='png') 

def dijkstra_fig(g, s):
  Q = PriorityQueue()
  S = []
  # task: vertex id, priority: shortest-path estimate, info: predecessor
  Q.add_task(task=s, priority=0, info=None)
  visited = set()
  v_pos = {'s': '0,0!', 't': '1,1!', 'x': '3, 1!', 'y': '1,-1!', 'z': '3,-1!'}
  idx = 0
  dot = Digraph(comment='directed_graph', engine="neato", format='png', strict=True, )
  plot_subgraph(dot, v_pos, g, Q, idx)
  while not Q.empty():
    # get all edges
    idx += 1
    # Use the light vertex
    # Update set S
    u, up, ud = Q.pop_task()
    dot.node(name=u, label=str(ud), xlabel = u, _attributes={'pos': v_pos[u], 'penwidth': "1.5", 'color': 'blue'})
     
    if up is not None:
      dot.edge(up, u, _attributes={'color': 'blue', 'penwidth': "1.5" })
    dot.render(f'test-output/dijkstra_{idx-1+0.5}', view=True, format='png') 

    visited.add(u)
    S.append((u, ud, up))

    # Relax adjacent vertice
    for v, w in g[u]: 
      # Already found the shortest path for this id
      if v in visited: 
        continue
      
      vd, vp = Q.get_task(v)
      # First time to add the task or already in the queue, but need update 
      if not vd  or ud + w < vd:
        Q.add_task(task=v, priority=ud + w, info=u)

        # return the old edge to normal
        if vd is not None and ud + w < vd:
          dot.edge(vp, v, _attributes={'color': 'black', 'penwidth': "1.5" })
        # Expand edges and return the old edges to normal
    plot_subgraph(dot, v_pos, g, Q, idx)        
  return S, dot

In [ ]:
dijkstra_fig(g, 's')

##All-pairs shortest paths
* Extension on Bellman-Ford algorithm
* Extension on Dijkstra's algorithm

##### Extended Bellman-ford algorithm

In [ ]:
# rewrite the non negative weight graph g as
g = {
    's':[('t', 6), ('y', 7)],
    't':[('x', 5), ('y', 8), ('z', -4)],
    'x':[('t',-2)],
    'y':[('x',-3), ('z',9)],
    'z':[('x',7)]    
}
n = len(g)
W = [[float('inf') for _ in range(n)] for _ in range(n)]
key2idx = {k : i for k, i in zip(g.keys(), range(n))}
idx2key = {i : k for k, i in zip(g.keys(), range(n))}
for u in g.keys():
  ui = key2idx[u]
  W[ui][ui] = 0
  for v, w in g[u]:
    vi = key2idx[v]
    W[ui][vi] =  w
W, key2idx, idx2key

In [ ]:
import copy
def bellman_ford(W, L):
  n = len(W)
  for i in range(n): # source
    for j in range(n): # endpoint
      for k in range(n): # extend one edge
        L[i][j] = min(L[i][j], L[i][k]+W[k][j])
  
def extended_bellman_ford(W):
  n = len(W)
  # initialize L, first pass
  L = copy.deepcopy(W)
  print(f'L1 : {L} \n')
  # n-2 passes
  for i in range(n-2):
    bellman_ford(W, L)
    print(f'L{i+2}: {L} \n')
  return L

In [ ]:
L = extended_bellman_ford(W)

Implement a version that tracks the predecessor


In [ ]:
import copy
def bellman_ford_with_predecessor(W, L, P):
  n = len(W)
  for i in range(n): # source
    for j in range(n): # endpoint
      for k in range(n): # extend one edge
        if L[i][k] + W[k][j] < L[i][j]:
          L[i][j] = L[i][k] + W[k][j] # set d
          P[i][j] = k # set predecessor
  
def extended_bellman_ford_with_predecessor(W):
  n = len(W)
  # initialize L, first pass
  L = copy.deepcopy(W)
  print(f'L1 : {L} \n')
  P = [[None for _ in range(n)] for _ in range(n)]
  for i in range(n):
    for j in range(n):
      if L[i][j] != 0 and L[i][j] != float('inf'):
        P[i][j] = i
  # n-2 passes
  for i in range(n-2):
    bellman_ford_with_predecessor(W, L, P)
    print(f'L{i+2}: {L} \n')
  return L, P

In [ ]:
# print the path from s-> u through backtracking starting from u
def print_path(P, s, u, path):
  path.append(u)
  if u == s:
    print('Reached to the source vertex, stop!')
    return path[::-1]
  elif u is None:
    print(f"No path found between {s} and {u}.")
    return []
  else:  
    return print_path(P, s, P[s][u], path)

In [ ]:
L, P = extended_bellman_ford_with_predecessor(W)
L, P

In [ ]:
# get path from s to t
path = print_path(P, key2idx['t'], key2idx['z'], [])
path = [idx2key[i] for i in path]
path

In [ ]:
#Find edges for each shortest_path tree
shortest_path_trees = [[] for _ in range(n)]
for i in range(n):
  for j in range(n):
    path = print_path(P, i, j, [])
    for u, v in zip(path[:-1], path[1:]):
      if (u, v) not in shortest_path_trees[i]:
        shortest_path_trees[i].append((u, v))
# now replace it all to keys
shortest_path_tree_dic = {}
for i in range(len(shortest_path_trees)):
  s = idx2key[i]
  if s not in shortest_path_tree_dic:
    shortest_path_tree_dic[s] = []
  for u, v in shortest_path_trees[i]:
    shortest_path_tree_dic[s].append((idx2key[u], idx2key[v]))
shortest_path_tree_dic

In [ ]:
from google.colab.patches import cv2_imshow
import cv2

for idx, s in enumerate(list(shortest_path_tree_dic.keys())):
  # stric so that one edge wont be repeat but keep the last time's edit
  dot = Digraph(comment='directed_graph', engine="neato", format='png', strict=True)
  # Generate grapg
  v_pos = {'s': '0,0!', 't': '1,1!', 'x': '3, 1!', 'y': '1,-1!', 'z': '3,-1!'}
  for u, p in v_pos.items():
    dot.node(u, _attributes={'pos': str(p), 'fillcolor': "#d62728"})
    for v, w in g[u]:
      dot.edge(u, v, _attributes={'xlabel': str(w)})
  
  # Color one tree
  colors = ['red', 'green', 'yellow', 'blue', 'purple']
  dot.node(s, _attributes={'pos': str(v_pos[s]), 'color': colors[idx]})
  for x, y in shortest_path_tree_dic[s]:
    dot.edge(x, y, _attributes={'color': colors[idx]}  )

  dot.render(f'test-output/shortest_path_trees_{idx}', view=True) 
  img = cv2.imread(f'test-output/shortest_path_trees_{idx}.png')
  cv2_imshow(img)

##### Repeated Square 



In [165]:
import copy
import math
def bellman_ford_repeated_square(L):
  n = len(W)
  for i in range(n): # source
    for j in range(n): # endpoint
      for k in range(n): # double the extending length
        L[i][j] = min(L[i][j], L[i][k]+L[k][j])
  
def extended_bellman_ford_repeated_square(W):
  n = len(W)
  # initialize L, first pass
  L = copy.deepcopy(W)
  print(f'L1 : {L} \n')
  # log n passes
  for i in range(math.ceil(math.log(n))):
    bellman_ford_repeated_square(L)
    print(f'L{2^(i+1)}: {L} \n')
  return L

In [166]:
L = extended_bellman_ford_repeated_square(W)

L1 : [[0, 6, inf, 7, inf], [inf, 0, 5, 8, -4], [inf, -2, 0, inf, inf], [inf, inf, -3, 0, 9], [inf, inf, 7, inf, 0]] 

L3: [[0, 6, 4, 7, 2], [inf, 0, 3, 8, -4], [inf, -2, 0, 6, -6], [inf, -5, -3, 0, -9], [inf, 5, 7, 13, 0]] 

L0: [[0, 2, 4, 7, -2], [inf, 0, 3, 8, -4], [inf, -2, 0, 6, -6], [inf, -5, -3, 0, -9], [inf, 5, 7, 13, 0]] 



In [167]:
math.ceil(math.log(5))

2

#####The Floyd-Warshall Algorithm

In [172]:
def floyd_warshall(W):
  L = copy.deepcopy(W) #L0
  n = len(W)
  for k in range(n): # intermediate node
    for i in range(n): # start node
      for j in range(n): # end node
        L[i][j] = min(L[i][j], L[i][k] + L[k][j])
  print(L)
  return L

In [173]:
floyd_warshall(W)

[[0, 2, 4, 7, -2], [inf, 0, 3, 8, -4], [inf, -2, 0, 6, -6], [inf, -5, -3, 0, -9], [inf, 5, 7, 13, 0]]


[[0, 2, 4, 7, -2],
 [inf, 0, 3, 8, -4],
 [inf, -2, 0, 6, -6],
 [inf, -5, -3, 0, -9],
 [inf, 5, 7, 13, 0]]

##Exercies

1. 847. Shortest Path Visiting All Nodes (hard). Traveling salesman problem, but can be better with shortest path, breath-first, and dijkstra algorthm